In [1]:
import os
from pathlib import Path
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import applications

In [2]:
path = '../mergedData/'

image_gen = ImageDataGenerator(validation_split=0.3)
train_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(100, 100),batch_size=64, class_mode='categorical', subset='training')
val_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(100, 100),batch_size=64, class_mode='categorical', subset='validation')

test_generator = image_gen.flow_from_directory(
    path + "Test", target_size=(100, 100),batch_size=64, class_mode='categorical')

Found 27680 images belonging to 67 classes.
Found 11833 images belonging to 67 classes.
Found 16999 images belonging to 67 classes.


In [3]:
print(train_generator.class_indices)
print(len(train_generator.class_indices))

{'Apple': 0, 'Apricot': 1, 'Avocado': 2, 'Banana': 3, 'Beetroot': 4, 'Blueberry': 5, 'Cactus': 6, 'Cantaloupe': 7, 'Carambula': 8, 'Cauliflower': 9, 'Cherry': 10, 'Chestnut': 11, 'Clementine': 12, 'Cocos': 13, 'Corn': 14, 'Cucumber': 15, 'Dates': 16, 'Eggplant': 17, 'Fig': 18, 'Ginger': 19, 'Granadilla': 20, 'Grape': 21, 'Grapefruit': 22, 'Guava': 23, 'Hazelnut': 24, 'Huckleberry': 25, 'Kaki': 26, 'Kiwi': 27, 'Kohlrabi': 28, 'Kumquats': 29, 'Lemon': 30, 'Limes': 31, 'Lychee': 32, 'Mandarine': 33, 'Mango': 34, 'Mangostan': 35, 'Maracuja': 36, 'Melon': 37, 'Mulberry': 38, 'Nectarine': 39, 'Nut': 40, 'Onion': 41, 'Orange': 42, 'Papaya': 43, 'Passion': 44, 'Peach': 45, 'Pear': 46, 'Pepino': 47, 'Pepper': 48, 'Physalis': 49, 'Pineapple': 50, 'Pitahaya': 51, 'Plum': 52, 'Pomegranate': 53, 'Pomelo': 54, 'Potato': 55, 'Quince': 56, 'Rambutan': 57, 'Raspberry': 58, 'Redcurrant': 59, 'Salak': 60, 'Strawberry': 61, 'Tamarillo': 62, 'Tangelo': 63, 'Tomato': 64, 'Walnut': 65, 'Watermelon': 66}
67


In [4]:
base_model = applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,
) # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

In [6]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)

In [7]:
# Create new model on top
inputs = layers.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)

In [8]:
# Pre-trained Xception weights requires that input be normalized
# from (0, 255) to a range (-1., +1.), the normalization layer
# does the following, outputs = (inputs - mean) / sqrt(var)
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# Scale inputs to [-1, +1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

In [9]:
# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(67)(x)
model = keras.Model(inputs, outputs)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 67)               

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [11]:
history = model.fit(train_generator, epochs=2, validation_data=val_generator)

Epoch 1/2
433/433 [==============================] - 966s 2s/step - loss: 11.8621 - accuracy: 0.1178 - val_loss: 11.0378 - val_accuracy: 0.1211
Epoch 2/2
433/433 [==============================] - 801s 2s/step - loss: 12.5386 - accuracy: 0.1454 - val_loss: 11.9733 - val_accuracy: 0.1356


In [14]:
base_model.trainable = True
model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_generator, epochs=2, validation_data=val_generator)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 150, 150, 3)       0         
_________________________________________________________________
normalization (Normalization (None, 150, 150, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 67)               

In [20]:
results = model.evaluate(test_generator)

266/266 [==============================] - 438s 2s/step - loss: 12.2248 - accuracy: 0.1652
